In [1]:
import os
sep_local = os.path.sep
sep_local

'\\'

In [2]:
%env TF_KERAS = 1

env: TF_KERAS=1


In [3]:
# import sys
# sys.path.append('..' + sep_local + '..' + sep_local + '..') # For Windows import

In [4]:
import os
os.chdir('..' + sep_local + '..' + sep_local + '..') # For Linux import

In [5]:
# from tensorflow.keras.mixed_precision import experimental as mixed_precision
# policy = mixed_precision.Policy('mixed_float16')
# mixed_precision.set_policy(policy)
# print('Compute dtype: %s' % policy.compute_dtype)
# print('Variable dtype: %s' % policy.variable_dtype)

In [6]:
import tensorflow as tf
tf.__version__

'2.1.0'

In [7]:
dataset_name = 'dsprites'
import tensorflow_datasets as tfds

info = tfds.builder(dataset_name).info

print(info)

tfds.core.DatasetInfo(
    name='dsprites',
    version=0.1.0,
    description='dSprites is a dataset of 2D shapes procedurally generated from 6 ground truth
independent latent factors. These factors are *color*, *shape*, *scale*,
*rotation*, *x* and *y* positions of a sprite.

All possible combinations of these latents are present exactly once,
generating N = 737280 total images.

### Latent factor values

*   Color: white
*   Shape: square, ellipse, heart
*   Scale: 6 values linearly spaced in [0.5, 1]
*   Orientation: 40 values in [0, 2 pi]
*   Position X: 32 values in [0, 1]
*   Position Y: 32 values in [0, 1]

We varied one latent at a time (starting from Position Y, then Position X, etc),
and sequentially stored the images in fixed order.
Hence the order along the first dimension is fixed and allows you to map back to
the value of the latents corresponding to that image.

We chose the latents values deliberately to have the smallest step changes
while ensuring that all pixel outp

In [ ]:
import tensorflow_datasets as tfds

info = tfds.builder(dataset_name).info

print(info)

In [ ]:
import tensorflow as tf

In [ ]:
train_ds = tf.data.Dataset.from_generator(
    lambda: training_generator, 
    output_types=tf.float32 ,
    output_shapes=tf.TensorShape((batch_size, ) + image_size)
)

test_ds = tf.data.Dataset.from_generator(
    lambda: testing_generator, 
    output_types=tf.float32 ,
    output_shapes=tf.TensorShape((batch_size, ) + image_size)
)


In [8]:
intermediate_dim = 20
inputs_shape=(64, 64, 1) # image shape
batch_size = 100
latents_dim = 6

In [10]:
TRAIN_BUF = 600
TEST_BUF = 100

import tensorflow_datasets as tfds
# Construct a tf.data.Dataset
train_ds = tfds.load(name=dataset_name, split=tfds.Split.TRAIN).shuffle(TRAIN_BUF).batch(batch_size)
try:
    test_ds = tfds.load(name=dataset_name, split=tfds.Split.TEST).shuffle(TEST_BUF).batch(batch_size)
except:
    test_ds = tfds.load(name=dataset_name, split=tfds.Split.TRAIN).shuffle(TEST_BUF).batch(batch_size)


In [11]:
_instance_scale=1.0
for data in train_ds:
    _instance_scale = float(data['image'][0].numpy().max())
    break

In [12]:
enc_lays2 = [
    tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=(2, 2), activation='relu'),
    tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=(2, 2), activation='relu'),
    tf.keras.layers.Flatten(),
    # No activation
    tf.keras.layers.Dense(latents_dim, dtype='float32')
]

dec_lays2 = [
    tf.keras.layers.Dense(units=16*16*32, activation=tf.nn.relu),
    tf.keras.layers.Reshape(target_shape=(16, 16, 32)),
    tf.keras.layers.Conv2DTranspose(filters=64, kernel_size=3, strides=(2, 2), padding="SAME", activation='relu'),
    tf.keras.layers.Conv2DTranspose(filters=32, kernel_size=3, strides=(2, 2), padding="SAME", activation='relu'),
    
    # No activation
    tf.keras.layers.Conv2DTranspose(filters=1, kernel_size=3, strides=(1, 1), padding="SAME", dtype='float32')
]

In [13]:
enc_mean_lays = [tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim, activation='relu')]

enc_var_lays = [tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim, activation='relu')]

dec_lays = [tf.keras.layers.Dense(units=2*intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=2*intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=2*intermediate_dim, activation='relu')]

In [14]:
#from utils.data_and_files.file_utils import make_random_string
#from time import gmtime, strftime

#model_name = 'AE_' + make_random_string(5) + strftime("%a_%d_%b_%Y_%H_%M", gmtime())
#print(model_name)

In [15]:
import os
model_name = dataset_name + 'Dense' +'AE'
#experiments_dir='..'+sep_local+'..'+sep_local+'..'+sep_local+'experiments'+sep_local + model_name
os.getcwd()
experiments_dir=os.getcwd()+ sep_local  +'experiments'+sep_local + model_name

In [16]:
from os.path import abspath
absolute = abspath(experiments_dir)
print("Recording_dir",absolute)
print("Current working dir",os.getcwd())

Recording_dir C:\Users\Khalid\Documents\projects\GM\Generative_Models\experiments\dspritesDenseAE
Current working dir C:\Users\Khalid\Documents\projects\GM\Generative_Models


In [17]:
from training.autoencoding_basic.autoencoders.autoencoder import autoencoder as AE

In [18]:
variables_params = \
[
    {
        'name': 'inference', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latents_dim,
        'layers':enc_lays2 #enc_mean_lays
    }
    ,
    
        {
        'name': 'generative', 
        'inputs_shape':latents_dim,
        'outputs_shape':inputs_shape,
        'layers':dec_lays2 #dec_lays
    }
]

In [19]:
from utils.data_and_files.file_utils import create_if_not_exist
_restore = os.path.join(experiments_dir, 'var_save_dir')
create_if_not_exist(_restore)
absolute = abspath(_restore)
print("Restore_dir",absolute)

Restore_dir C:\Users\Khalid\Documents\projects\GM\Generative_Models\experiments\dspritesDenseAE\var_save_dir


In [20]:
ae = AE( 
    name=model_name,
    latents_dim=latents_dim,
    batch_size=batch_size,
    variables_params=variables_params, 
    filepath=None #to restore trained model, set filepath=_restore
    )

Model: "inference"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_inputs (InputLayer [(None, 64, 64, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 31, 31, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 15, 15, 64)        18496     
_________________________________________________________________
flatten (Flatten)            (None, 14400)             0         
_________________________________________________________________
dense (Dense)                (None, 6)                 86406     
_________________________________________________________________
batch_normalization (BatchNo (None, 6)                 24        
_________________________________________________________________
dropout (Dropout)            (None, 6)                 0 

  WARNING  | None


Model: "generative"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
generative_inputs (InputLaye [(None, 6)]               0         
_________________________________________________________________
dense_1 (Dense)              (None, 8192)              57344     
_________________________________________________________________
reshape (Reshape)            (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 32, 32, 64)        18496     
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 64, 64, 32)        18464     
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 64, 64, 1)         289       
_________________________________________________________________
batch_normalization_1 (Batch (None, 64, 64, 1)         4

  WARNING  | None


In [21]:
#ae.compile(metrics=None)
ae.compile()

Model: "dspritesDenseAE"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_inputs (InputLayer [(None, 64, 64, 1)]       0         
_________________________________________________________________
inference (Model)            (None, 6)                 105246    
_________________________________________________________________
generative (Model)           (None, 64, 64, 1)         94597     
_________________________________________________________________
tf_op_layer_x_logits (Tensor [(None, 64, 64, 1)]       0         
Total params: 199,843
Trainable params: 199,829
Non-trainable params: 14
_________________________________________________________________
None


In [22]:
import logging
mpl_logger = logging.getLogger('matplotlib')
mpl_logger.setLevel(logging.WARNING)

from training.callbacks.sample_generation import SampleGeneration
from training.callbacks.save_model import ModelSaver

In [23]:
es = tf.keras.callbacks.EarlyStopping(
    monitor='loss', 
    min_delta=1e-12, 
    patience=5, 
    verbose=1, 
    restore_best_weights=True
)

In [24]:
ms = ModelSaver(filepath=_restore,save_freq=5)

In [25]:
csv_dir = os.path.join(experiments_dir, 'csv_dir')
create_if_not_exist(csv_dir)
csv_dir = os.path.join(csv_dir, model_name+'.csv')
csv_log = tf.keras.callbacks.CSVLogger(csv_dir, append=True)
absolute = abspath(csv_dir)
print("Csv_dir",absolute)

Csv_dir C:\Users\Khalid\Documents\projects\GM\Generative_Models\experiments\dspritesDenseAE\csv_dir\dspritesDenseAE.csv


In [26]:
image_gen_dir = os.path.join(experiments_dir, 'image_gen_dir')
create_if_not_exist(image_gen_dir)
absolute = abspath(image_gen_dir)
print("Image_gen_dir",absolute)

Image_gen_dir C:\Users\Khalid\Documents\projects\GM\Generative_Models\experiments\dspritesDenseAE\image_gen_dir


In [27]:
sg = SampleGeneration(latents_shape=6, filepath=image_gen_dir, gen_freq=5, save_img=True, gray_plot=True)

In [28]:
#DATA_DOWN_PATH = '..'+sep_local+'..'+sep_local+'..'+sep_local+'data'
DATA_DOWN_PATH = os.getcwd() + sep_local+'data'
Script_dir = os.getcwd() + sep_local+'data'+sep_local+'download_gt_data.sh'
# Script call to download "dsprites_full" dataset_name 
!/bin/bash $Script_dir -f $DATA_DOWN_PATH -d $dataset_name

The system cannot find the path specified.


In [29]:
from data.gt_load.datasets import load
DATA_PATH = DATA_DOWN_PATH +sep_local+'.gt_datasets'
absolute = abspath(DATA_PATH)
print("DATA_PATH",absolute)

DATA_PATH C:\Users\Khalid\Documents\projects\GM\Generative_Models\data\.gt_datasets


In [30]:
DATA_PATH

'C:\\Users\\Khalid\\Documents\\projects\\GM\\Generative_Models\\data\\.gt_datasets'

In [31]:
#! mkdir "C:\Users\Khalid\Documents\projects\GM\Generative_Models\data\.gt_datasets\dsprites"

In [32]:
#! wget -O 'C:\Users\Khalid\Documents\projects\GM\Generative_Models\data\.gt_datasets\dsprites\dsprites_ndarray_co1sh3sc6or40x32y32_64x64.npz' https://github.com/deepmind/dsprites-dataset/raw/master/dsprites_ndarray_co1sh3sc6or40x32y32_64x64.npz

In [33]:
eval_dataset = load(dataset_name='dsprites_full', dataset_path=DATA_PATH)

In [34]:
gts_csv = os.path.join(experiments_dir, 'csv_dir', 'gts_metrics')
gtu_csv = os.path.join(experiments_dir, 'csv_dir', 'gtu_metrics')

In [35]:
import numpy as np

In [36]:
from training.callbacks.disentangle_supervied import DisentanglementSuperviedMetrics
from training.callbacks.disentangle_unsupervied import DisentanglementUnsuperviedMetrics

In [37]:
gts_mertics = DisentanglementSuperviedMetrics(            
    ground_truth_data=eval_dataset,
    representation_fn=lambda x: ae.encode(x),
    random_state=np.random.RandomState(0),
    file_Name=gts_csv,
    num_train=1000,
    num_test=200,
    batch_size=batch_size,
    continuous_factors=False,
    gt_freq=2
)
gtu_mertics = DisentanglementUnsuperviedMetrics(            
    ground_truth_data=eval_dataset,
    representation_fn=lambda x: ae.encode(x),
    random_state=np.random.RandomState(0),
    file_Name=gtu_csv,
    num_train=1000,
    num_test=200,
    batch_size=batch_size,
    gt_freq=2
)

In [38]:
ae.fit(
    x=train_ds,
    steps_per_epoch=10,#1000,
    epochs=int(1e6), 
    verbose=2,
    callbacks=[ es, ms, csv_log, sg, gts_mertics, gtu_mertics],
    workers=-1,
    use_multiprocessing=True
)

Train for 10 steps
Epoch 1/1000000


C:\ProgramData\Anaconda3\envs\tf02\lib\site-packages\dask\array\core.py:1333: FutureWarning: The `numpy.expand_dims` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  FutureWarning,
C:\ProgramData\Anaconda3\envs\tf02\lib\site-packages\dask\array\core.py:1333: FutureWarning: The `numpy.expand_dims` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  FutureWarning,
C:\ProgramData\Anaconda3\envs\tf02\lib\site-packages\dask\array\core.py:1333: FutureWarning: The `numpy.expand_dims` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  FutureWarning,
C:\ProgramData\Anaconda3\envs\tf02\lib\site-pa

10/10 - 27s - loss: 1518.8842 - psnr: 9.8653 - ssmi: 0.0013 - sharpdiff: 4.9218
Epoch 2/1000000
10/10 - 1s - loss: 1393.1567 - psnr: 10.3049 - ssmi: 0.0010 - sharpdiff: 4.2548
Epoch 3/1000000


C:\ProgramData\Anaconda3\envs\tf02\lib\site-packages\dask\array\core.py:1333: FutureWarning: The `numpy.expand_dims` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  FutureWarning,
C:\ProgramData\Anaconda3\envs\tf02\lib\site-packages\dask\array\core.py:1333: FutureWarning: The `numpy.expand_dims` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  FutureWarning,
C:\ProgramData\Anaconda3\envs\tf02\lib\site-packages\dask\array\core.py:1333: FutureWarning: The `numpy.expand_dims` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  FutureWarning,
C:\ProgramData\Anaconda3\envs\tf02\lib\site-pa

10/10 - 22s - loss: 1380.9624 - psnr: 10.3262 - ssmi: 9.7521e-04 - sharpdiff: 4.2576
Epoch 4/1000000
10/10 - 1s - loss: 1387.2617 - psnr: 10.3090 - ssmi: 8.6916e-04 - sharpdiff: 4.2608
Epoch 5/1000000


...

10/10 - 22s - loss: 981.4804 - psnr: 10.8946 - ssmi: 0.0102 - sharpdiff: 4.3076
Epoch 16/1000000
10/10 - 1s - loss: 982.4940 - psnr: 10.9115 - ssmi: 0.0110 - sharpdiff: 4.3079
Epoch 17/1000000


C:\ProgramData\Anaconda3\envs\tf02\lib\site-packages\dask\array\core.py:1333: FutureWarning: The `numpy.expand_dims` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  FutureWarning,
C:\ProgramData\Anaconda3\envs\tf02\lib\site-packages\dask\array\core.py:1333: FutureWarning: The `numpy.expand_dims` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  FutureWarning,
C:\ProgramData\Anaconda3\envs\tf02\lib\site-packages\dask\array\core.py:1333: FutureWarning: The `numpy.expand_dims` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  FutureWarning,
C:\ProgramData\Anaconda3\envs\tf02\lib\site-pa

10/10 - 22s - loss: 978.4468 - psnr: 10.9277 - ssmi: 0.0116 - sharpdiff: 4.3093
Epoch 18/1000000
10/10 - 1s - loss: 970.9592 - psnr: 10.9486 - ssmi: 0.0121 - sharpdiff: 4.3028
Epoch 19/1000000


C:\ProgramData\Anaconda3\envs\tf02\lib\site-packages\dask\array\core.py:1333: FutureWarning: The `numpy.expand_dims` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  FutureWarning,
C:\ProgramData\Anaconda3\envs\tf02\lib\site-packages\dask\array\core.py:1333: FutureWarning: The `numpy.expand_dims` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  FutureWarning,
C:\ProgramData\Anaconda3\envs\tf02\lib\site-packages\dask\array\core.py:1333: FutureWarning: The `numpy.expand_dims` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  FutureWarning,
C:\ProgramData\Anaconda3\envs\tf02\lib\site-pa

KeyboardInterrupt: 